In [58]:
import pandas as pd
import numpy as np
import os 
from typing import Dict, Any



def convert_1m_csv_to_5m_df(file_path: str) -> pd.DataFrame:
    # Column Names (သင့်ရဲ့ Data အစီအစဉ်အတိုင်း)
    COLUMN_NAMES = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    DATETIME_FORMAT = '%Y.%m.%d %H:%M'

    """ CSV File မှ 1-Minute Data ကို Load ပြီး 5-Minute Candle သို့ ပြောင်းလဲသည်။ """
    if not os.path.exists(file_path):
        print(f"🚨 Error: File not found at path: {file_path}")
        return pd.DataFrame()

    try:
        df = pd.read_csv(file_path, sep=',', header=None, names=COLUMN_NAMES,
                         dtype={'Open': np.float64, 'High': np.float64, 'Low': np.float64, 'Close': np.float64})
        
        # Volume column ကို ယာယီ float အနေဖြင့် သတ်မှတ်ပြီး NA များကို 0 ဖြင့် အစားထိုးသည်။
        df['Volume'] = pd.to_numeric(df['Volume'], errors='coerce').fillna(0)
        df['Volume'] = df['Volume'].astype(np.int64) 
        
    except Exception as e:
        print(f"🚨 Error loading CSV file: {e}")
        return pd.DataFrame()

    # Datetime Index ကို တည်ဆောက်ခြင်း
    df['Datetime'] = df['Date'].astype(str) + ' ' + df['Time'].astype(str)
    df['Datetime'] = pd.to_datetime(df['Datetime'], format=DATETIME_FORMAT, errors='coerce')
    df.set_index('Datetime', inplace=True)
    df.drop(columns=['Date', 'Time'], inplace=True)
    #df.dropna(subset=[df.index.name], inplace=True) # Invalid Datetime များကို ဖယ်ရှားခြင်း

    ohlcv_aggregation_rules: Dict[str, Any] = {
        'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'
    }
    df_5m = df.resample('5Min').agg(ohlcv_aggregation_rules)
    df_5m.dropna(inplace=True)
    df_5m = df_5m[df_5m['Volume'] > 0]
    
    print(f"✅ Conversion successful! 5-Min rows: {len(df_5m)}")
    return df_5m

In [77]:
import json

class EnvConfig():
    """environment configuration from json file
       tgym requires you configure your own parameters in json file.
        Args:
            config_file path/file.json

    """
    def __init__(self,config_file):
        self.config = {}
        with open(config_file) as j:
            self.config = json.load(j)

    def env_parameters(self,item=''):
        """environment variables
        """
        if item:
            return self.config["env"][item]
        else:
            return self.config["env"]

    def symbol(self, asset="GBPUSD", item='') :
        """get trading pair (symbol) information

        Args:
            asset (str, optional): symbol in config. Defaults to "GBPUSD".
            item (str, optional): name of item, if '' return dict, else return item value. Defaults to ''.

        Returns:
            [type]: [description]
        """
        if item:
            return self.config["symbol"][asset][item]
        else:
            return self.config["symbol"][asset]

    def data_processing_parameters(self, item=''):
        """Get data processing config"""
        if item:
            return self.config["data_processing"][item]
        return self.config["data_processing"]

    def trading_hour(self,place="NewYork"):
        """forex trading hour from different markets

        Args:
            place (str, optional): [Sydney,Tokyo,London] Defaults to "New York".

        Returns:
            [dict]: from time, to time
        """
        if place:
            return self.config["trading_hour"][place]
        else:
            return self.config["trading_hour"]
        
    def indicator(self,place="sma_fast_period"):
        """forex trading hour from different markets

        Args:
            place (str, optional): [Sydney,Tokyo,London] Defaults to "New York".

        Returns:
            [dict]: from time, to time
        """
        if place:
            return self.config["data_processing"]["indicator"][place]
        else:
            return self.config["data_processing"]["indicator"]



In [62]:

import pandas as pd
import logging
from finta import TA
from sklearn.preprocessing import StandardScaler
import holidays
import os
import json

import logging
# Configure logging
logger = logging.getLogger(__name__)

def patch_missing_data(df, dt_col_name='time', cf=None):
    min_bars = cf.data_processing_parameters("min_bars_per_week")

    # ["time","open", "high", "low", "close"]
    required_cols = cf.data_processing_parameters("required_cols")

    # df မှာ 6 columns ရှိရင် vol ပါထည့်မယ်
    if df.shape[1] == 6:
        df.columns = required_cols + ['vol']
    elif df.shape[1] == 5:
        df.columns = required_cols
    else:
        raise ValueError(f"Invalid number of columns: {df.shape[1]} =>{required_cols}")

    logger.warning(f"shape of  column: {df.shape[1]}")
    # 1. Column validation
    if missing := set(required_cols) - set(df.columns):
        raise ValueError(f"Missing columns: {missing}")

    # 2. Auto-detect datetime column
    dt_candidates = {'time', 'timestamp', 'date', 'datetime'}
    if dt_col_name not in df.columns:
        found = list(dt_candidates & set(df.columns))
        if not found:
            raise KeyError(f"No datetime column found. Tried: {dt_candidates}")
        dt_col_name = found[0]
        logger.info(f"Using datetime column: {dt_col_name}")

    # 3. Convert to datetime index
    df[dt_col_name] = pd.to_datetime(df[dt_col_name], utc=True)
    df = df.set_index(dt_col_name).sort_index()
    groups = df.groupby(pd.Grouper(freq='W-SUN'))

    patched_weeks = []  # patched weekly df storage

    for w, week_df in groups:
        if week_df.empty:
            continue

        if len(week_df) != min_bars:
            logger.warning(f"Week {w} has {len(week_df)}/{min_bars} bars")

        # Create 5-minute frequency index
        new_index = pd.date_range(
            start=week_df.index.min(),
            end=week_df.index.max(),
            freq='5min',
            tz='UTC'
        )

        # Reindex + forward fill
        week_df = week_df.reindex(new_index)
        week_df.index = week_df.index.tz_localize(None)
        fill_limit = 12 # ဥပမာ: 1 နာရီ (12 bars) ထက်ပိုတဲ့ ကွက်လပ်ကို မဖြည့်ပါ
        fill_cols = ['open', 'high', 'low', 'close', 'vol'] if 'vol' in df.columns else ['open', 'high', 'low', 'close']
        # FFill: ရှေ့က data ဖြင့် ဖြည့်ပါ
        week_df[fill_cols] = week_df[fill_cols].ffill(limit=fill_limit)
        patched_weeks.append(week_df)

    # Merge back all weeks
    if patched_weeks:
        all_df = pd.concat(patched_weeks)
    else:
        all_df = df.copy()

    return all_df

In [ ]:

def add_time_feature(df_5m: pd.DataFrame, cf=None, source_tz='UTC') -> pd.DataFrame:
    """
    5M Data Frame (DatetimeIndex ပါဝင်သည်ဟု ယူဆပါ) အတွက် Temporal features ထည့်သွင်းခြင်း။
    """
    
    # DataFrame ၏ Index ကို DatetimeIndex အဖြစ် စစ်ဆေးခြင်း
    if not isinstance(df_5m.index, pd.DatetimeIndex):
         raise TypeError("DataFrame ၏ Index သည် DatetimeIndex ဖြစ်ရပါမည်။")

    df_5m.index = df_5m.index.tz_localize(None)
    # Index ကို Timezone aware (UTC) အဖြစ် သေချာအောင်လုပ်ခြင်း
    if df_5m.index.tz is None:
        # Timezone-Naive data ကို မူရင်း Source Timezone ဖြင့် localize
        # Dukascopy data ဆိုရင် 'UTC' သုံးပြီး၊ Broker data ဆိုရင် 'GMT+3' လိုမျိုး သုံးနိုင်ပါတယ်
        df = df_5m.tz_localize(source_tz, ambiguous='NaT', nonexistent='NaT')
        df = df.tz_convert('UTC')
    else:
        df = df_5m.copy()
    
    # ----------------------------------------------------
    # I. အခြေခံ features နှင့် Cyclical Encoding များ (Hour ကို Index မှ တိုက်ရိုက်ယူခြင်း)
    # ----------------------------------------------------

    df['weekday'] = df.index.dayofweek 
    df['day'] = df.index.day
    df['week'] = df.index.isocalendar().week.astype(int)
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['hour'] = df.index.hour
    
    # နာရီအတွက် Cyclical features
    df['hour_sin'] = np.sin(2 * np.pi * df['hour']/24).round(6)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour']/24).round(6)

    # ----------------------------------------------------
    # III. DST-Aware Market Sessions (Timezone Handling)
    # ----------------------------------------------------
    
    # နာရီကို local time zone သို့ ပြောင်းလဲ (Timezone Aware Index မှသာ tz_convert လုပ်နိုင်သည်)
    london_time = df.index.tz_convert('Europe/London')
    ny_time = df.index.tz_convert('America/New_York')

    # Session Hours (cf မှ Local Time နာရီများကို ပေးပို့ရပါမည်။)
    ny = cf.trading_hour('NewYork')
    ldn = cf.trading_hour('London')

    # London Session (Local Time: 08:00 - 16:00)
    df['london_session'] = ((london_time.hour >= ldn['from']) & (london_time.hour < ldn['to'])).astype(int)
    
    # NY Session (Local Time: 13:00 - 21:00 UTC/GMT) -> (9:00 - 17:00 EST/EDT)
    # cf မှ Local NY Time (ဥပမာ: 9, 17) ကို ပေးပို့ရမည်
    df['ny_session'] = ((ny_time.hour >= ny['from']) & (ny_time.hour < ny['to'])).astype(int)

    df['overlap_session'] = (df['london_session'] & df['ny_session']).astype(int)

    # ... (IV. Holiday features ကို ဆက်လက်ထည့်သွင်းနိုင်သည်) ...
    
    #df['symbol'] = symbol
    
    # ယာယီ columns များကို ဖယ်ရှားခြင်း
    df = df.drop(columns=['minute_block_15'], errors='ignore') # minute_block_15 သည် 1M data မှ လာလျှင်သာ လိုအပ်သည်။ 5M တွင် မလိုအပ်ပါ။
    
    # Index ကို reset မလုပ်ဘဲ ပြန်ပို့ပါ (Env အတွက် Datetime Index လိုအပ်သည်)
    return df

In [ ]:
from finta import TA

def tech_indicators(df, cf=None):
    """
    Forex RL အတွက် Price Action၊ Momentum နှင့် Long-Term Trend Features များကို ထည့်သွင်းခြင်း။
    """
    
    sma_fast_period = cf.indicator('sma_fast_period')
    sma_mid_period = cf.indicator('sma_mid_period')
    sma_slow_period = cf.indicator('sma_slow_period')
    atr_period = cf.indicator('atr_period')
    rsi_period = cf.indicator('rsi_period')
    macd_fast_period = cf.indicator('macd_fast_period')
    macd_slow_period = cf.indicator('macd_slow_period')
    macd_signal_period = cf.indicator('macd_signal_period')
    adx_period = cf.indicator('adx_period')
    stoch_period = cf.indicator('stoch_period')

    
    # --- ၁။ Volatility Measure (ATR ကို Base အဖြစ် အသုံးပြုရန်) ---
    df['atr_base'] = TA.ATR(df, period=atr_period).ffill()

    # --- ၂။ Price Action Features ---
    df['log_returns'] = np.log(df['close'] / df['close'].shift(1)).ffill().round(6)
    df['price_norm'] = (df['close'] - df['close'].shift(sma_fast_period)) / df['atr_base'] # 100-bar SMA ကို ခန့်မှန်းခြေသုံး
    df['spread_ratio'] = (df['high'] - df['low']) / df['atr_base']
    df['body_ratio'] = (df['close'] - df['open']) / (df['high'] - df['low']).replace(0, 1e-6)

    # --- ၃။ Momentum & Trend Features (SMA 100/200 Cross ကို ထည့်သွင်းခြင်း) ---

    df['rsi'] = TA.RSI(df, period=rsi_period).ffill().round(6)
    macd_data = TA.MACD(df, period_fast=macd_fast_period, period_slow=macd_slow_period, signal=macd_signal_period)
    df['macd_hist'] = macd_data.SIGNAL.ffill().round(6)
    
    # Trend ရှိခြင်း/မရှိခြင်း နှင့် Trend ၏ ခိုင်မာမှု ကို တိုင်းတာခြင်း။
    # SMA Cross က Trend Direction ကို ပေးသော်လည်း၊ 
    # ADX က Direction ရဲ့ Strength ကို ပေးသည်။ 
    # ADX နိမ့်ပါက Range/Sideways ဖြစ်ပြီး၊ 
    # ADX မြင့်ပါက Strong Trend ဖြစ်ကြောင်း Agent ကို သိစေသည်။
    df['adx'] = TA.ADX(df, period=adx_period)  
    
    # RSI ဖြင့် Overbought/Oversold ကို စစ်ဆေးပြီး၊ 
    # Stochastic ဖြင့် Momentum Change ကို အတည်ပြုနိုင်သည်။ 
    # နှစ်ခုစလုံး Extreme Level တွင် ရှိနေပါက Reversal ဖြစ်နိုင်ခြေ ပိုများကြောင်း Agent က သင်ယူနိုင်သည်။
    df['stoch_k'] = TA.STOCH(df, period=stoch_period)   
    # *** သင်တောင်းဆိုသော Long-Term SMA Cross Feature အသစ် ***

    ma_mid = TA.SMA(df, period=sma_mid_period)
    ma_slow = TA.SMA(df, period=sma_slow_period)
    
    # SMA 100 နှင့် SMA 200 တို့၏ ကွာခြားချက်ကို ATR ဖြင့် Normalization လုပ်ခြင်း
    df['ma_cross'] = ((ma_mid - ma_slow) / df['atr_base']).ffill().round(6)
    
    # --- ၄။ Data Cleaning ---
    df = df.replace([np.inf, -np.inf], np.nan).fillna(0) 

    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df[numeric_cols] = df[numeric_cols].clip(lower=-1e5, upper=1e5).round(6)
    
    return df

In [ ]:
file_path_from_data = "./drive/MyDrive/data/raw/EURUSD_2020_m1_all.csv"
file_path_to_data = "./drive/MyDrive/data/raw/EURUSD_2020_m5_all.csv"

df_all_5m_data = convert_1m_csv_to_5m_df(file_path_from_data)
df_all_5m_data.to_csv(file_path_to_data)


cf = EnvConfig('./drive/MyDrive/configure.json')

raw = pd.read_csv(file_path_to_data)

df = patch_missing_data(raw,cf=cf)

symbol = 'EURUSD'
# Broker Data (00:00 မှ စသော) ကို ခေါ်ဆိုသည့်အခါ
# GMT+2/GMT+3 ကို အလိုအလျောက် ကိုင်တွယ်ပေးရန်
axiory_tz = 'Europe/Kiev'  

dft = add_time_feature(df, cf=cf, source_tz=axiory_tz)
dft.head(3)



cf = EnvConfig('./drive/MyDrive/configure.json')

df = tech_indicators(dft, cf=cf)
df.to_csv('tech.csv')